## Task 2 : Automated Hyperparameter Search

In [1]:
from autogluon.common import space

In [2]:
import os
import random
import numpy as np
import pandas as pd
from itertools import product
from autogluon.tabular import TabularPredictor
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Load dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)  # Reshape for one-hot encoding

# One-hot encoding
ohe = OneHotEncoder(sparse_output=False)
y_onehot = ohe.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_onehot, test_size=0.2, random_state=42)

# Convert to DataFrames
train_df = pd.DataFrame(X_train, columns=["feature_0", "feature_1", "feature_2", "feature_3"])
train_df["target"] = np.argmax(y_train, axis=1)

test_df = pd.DataFrame(X_test, columns=["feature_0", "feature_1", "feature_2", "feature_3"])
test_df["target"] = np.argmax(y_test, axis=1)

# Ensure logs directory exists
os.makedirs("logs", exist_ok=True)

/Users/aryan/Desktop/Academics /Semester 4/AI Software tools and techniques/-CS-203-MLP-Model-Implementation-Experiment-Tracking-Hyperparameters/venv_alt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def train_model(model_name, nn_options, train_df, metric, log_file):
    """Trains AutoGluon's TabularPredictor and logs results."""
    predictor = TabularPredictor(label='target', 
                                 path=f'./models/{model_name}', 
                                 eval_metric=metric).fit(train_df, 
                                                         hyperparameters={'NN_TORCH': nn_options})
    
    # Evaluate
    predictions = predictor.predict(test_df)
    y_true = test_df["target"].values
    acc = accuracy_score(y_true, predictions)
    f1 = f1_score(y_true, predictions, average='weighted')

    # Log results
    log_entry = f"Model: {model_name} | LR: {nn_options['alpha']} | Epochs: {nn_options['num_epochs']} | Acc: {acc:.4f} | F1: {f1:.4f}\n"
    with open(log_file, "a") as f:
        f.write(log_entry)
    
    return {"Model": model_name, "LR": nn_options["alpha"], "Epochs": nn_options["num_epochs"], "Accuracy": acc, "F1": f1}

In [4]:
def grid_search(train_df, metric):
    learning_rates = [1e-3, 1e-5]
    epochs = [1, 3, 5]
    log_file = f"logs/grid_search_{metric}.log"
    
    results = []
    for lr, epoch in product(learning_rates, epochs):
        nn_options = {
            'num_epochs': epoch,
            'alpha': lr,
            'activation': space.Categorical('relu'),
            'num_layers': 1,
            'hidden_size': 16
        }
        model_name = f"grid_{metric}_lr{lr}_ep{epoch}"
        results.append(train_model(model_name, nn_options, train_df, metric, log_file))
    
    return pd.DataFrame(results)

In [5]:
def random_search(train_df, metric, num_samples=3):
    learning_rates = [1e-3, 1e-5]
    epochs = [1, 3, 5]
    log_file = f"logs/random_search_{metric}.log"
    
    results = []
    for _ in range(num_samples):
        lr = random.choice(learning_rates)
        epoch = random.choice(epochs)
        
        nn_options = {
            'num_epochs': epoch,
            'alpha': lr,
            'activation': space.Categorical('relu'),
            'num_layers': 1,
            'hidden_size': 16
        }
        model_name = f"random_{metric}_lr{lr}_ep{epoch}"
        results.append(train_model(model_name, nn_options, train_df, metric, log_file))
    
    return pd.DataFrame(results)

In [6]:
def hyperband_search(train_df, metric):
    learning_rates = [1e-3, 1e-5]
    max_epochs = 5
    log_file = f"logs/hyperband_{metric}.log"
    
    results = []
    candidates = [{"alpha": lr, "num_epochs": max_epochs} for lr in learning_rates]
    
    for round in range(3):  # Simulating Hyperband rounds
        for candidate in candidates:
            nn_options = {
                'num_epochs': candidate["num_epochs"],
                'alpha': candidate["alpha"],
                'activation': space.Categorical('relu'),
                'num_layers': 1,
                'hidden_size': 16
            }
            model_name = f"hyperband_{metric}_lr{candidate['alpha']}_ep{candidate['num_epochs']}"
            results.append(train_model(model_name, nn_options, train_df, metric, log_file))
        
        # Reduce epochs for next round
        for candidate in candidates:
            candidate["num_epochs"] = max(1, candidate["num_epochs"] // 2)
    
    return pd.DataFrame(results)

In [7]:
def bayesian_search(train_df, metric, num_iters=5):
    log_file = f"logs/bayesian_search_{metric}.log"
    
    results = []
    prev_results = []  # Store previous scores to guide search
    
    for _ in range(num_iters):
        if prev_results:
            # Pick best-performing LR from past runs
            best_lr = max(prev_results, key=lambda x: x['Accuracy'])['LR']
            lr = best_lr if random.random() < 0.7 else random.choice([1e-3, 1e-5])
        else:
            lr = random.choice([1e-3, 1e-5])
        
        epoch = random.choice([1, 3, 5])
        
        nn_options = {
            'num_epochs': epoch,
            'alpha': lr,
            'activation': space.Categorical('relu'),
            'num_layers': 1,
            'hidden_size': 16
        }
        model_name = f"bayesian_{metric}_lr{lr}_ep{epoch}"
        result = train_model(model_name, nn_options, train_df, metric, log_file)
        results.append(result)
        prev_results.append(result)
    
    return pd.DataFrame(results)

In [9]:
# Run searches separately for 'accuracy' and 'f1'
metrics = ["accuracy","f1_micro"]
all_results = []

for metric in metrics:
    print(f"Running {metric} optimization...\n")
    
    df_grid = grid_search(train_df, metric)
    df_random = random_search(train_df, metric)
    df_hyperband = hyperband_search(train_df, metric)
    df_bayesian = bayesian_search(train_df, metric)
    
    all_results.append(pd.concat([df_grid, df_random, df_hyperband, df_bayesian], ignore_index=True))

# Combine results into a single DataFrame
final_results = pd.concat(all_results, keys=metrics, names=["Metric"])
final_results.to_csv("logs/hp_tuning_results.csv", index=False)
print("Hyperparameter tuning completed and logged!")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.16
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.1.0: Thu Oct 10 21:06:57 PDT 2024; root:xnu-11215.41.3~3/RELEASE_ARM64_T6041
CPU Count:          12
Memory Avail:       5.04 GB / 24.00 GB (21.0%)
Disk Space Avail:   325.13 GB / 460.43 GB (70.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high

Running accuracy optimization...



	0.0833	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 1.0}
	0.0833	 = Validation score   (accuracy)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.04s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 19451.8 rows/s (24 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/Users/aryan/Desktop/Academics /Semester 4/AI Software tools and techniques/-CS-203-MLP-Model-Implementation-Experiment-Tracking-Hyperparameters/models/grid_accuracy_lr1e-05_ep1")
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.16
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.1.0: Thu Oct 10 21:06:57 PDT 2024; root:xnu-11215.41.3~3/RELEASE_ARM64_T6041
CPU Cou

Running f1_micro optimization...



	0.0833	 = Validation score   (f1_micro)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 1.0}
	0.0833	 = Validation score   (f1_micro)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.06s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 9759.9 rows/s (24 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/Users/aryan/Desktop/Academics /Semester 4/AI Software tools and techniques/-CS-203-MLP-Model-Implementation-Experiment-Tracking-Hyperparameters/models/grid_f1_micro_lr1e-05_ep1")
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.16
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.1.0: Thu Oct 10 21:06:57 PDT 2024; root:xnu-11215.41.3~3/RELEASE_ARM64_T6041
CPU Coun

Hyperparameter tuning completed and logged!
